In [1]:

import requests
import json
import boto3
from botocore.exceptions import NoCredentialsError
import sys
import re

In [2]:

# AI Model Settings
INFERENCE_ENDPOINT = "https://granite-aiops.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com"
MODEL_API_URL = f"{INFERENCE_ENDPOINT}/v1/completions"
MODEL_NAME = "granite"

# MinIO Storage Settings
MINIO_ENDPOINT = "minio-api-aiops.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com"
MINIO_ACCESS_KEY = "minio"
MINIO_SECRET_KEY = "minio123"
MINIO_BUCKET = "logs"
LOG_FILE = "service_error.txt"

In [3]:

def read_log_from_minio(bucket_name, object_name):
    """Reads a log file's content from a MinIO bucket."""
    print(f"--> Step 1: Reading '{object_name}' from bucket '{bucket_name}'...")
    try:
        s3_client = boto3.client(
            's3',
            endpoint_url=f"http://{MINIO_ENDPOINT}",
            aws_access_key_id=MINIO_ACCESS_KEY,
            aws_secret_access_key=MINIO_SECRET_KEY
        )
        response = s3_client.get_object(Bucket=bucket_name, Key=object_name)
        content = response['Body'].read().decode('utf-8')
        print(f"    ✅ Successfully read log content from MinIO.")
        return content
    except Exception as e:
        print(f"    ❌ An error occurred while reading from MinIO: {e}")
    return None


In [4]:
read_log_from_minio(MINIO_BUCKET, LOG_FILE)

--> Step 1: Reading 'service_error.txt' from bucket 'logs'...
    ✅ Successfully read log content from MinIO.


'httpd.service - The Apache HTTP Server\n     Loaded: loaded (/usr/lib/systemd/system/httpd.service; enabled; vendor preset: disabled)\n     Active: inactive (dead) since Mon 2025-06-16 10:14:32 UTC; 5min ago\n       Docs: man:httpd.service(8)\n\nJun 16 10:14:32 server1 systemd[1]: Stopping The Apache HTTP Server...\nJun 16 10:14:32 server1 httpd[12345]: AH00015: SIGTERM received. Shutting down.\nJun 16 10:14:32 server1 systemd[1]: httpd.service: Succeeded.\nJun 16 10:14:32 server1 systemd[1]: Stopped The Apache HTTP Server.'

In [9]:
def query_model_for_report(logs):
    """Queries the AI model to generate a full incident report."""
    print(f"\n--> Step 2: Querying AI model for full report...")
    prompt = f"""
Generate a concise, structured AIOps incident report based on the following logs. The service name must be the short, executable name (e.g., 'httpd').
The report must be in Markdown format and include:
1. Key metadata: INCIDENT ID, DETECTED, SEVERITY, STATUS, AFFECTED SERVICE, AFFECTED HOST, SUMMARY, and KEY ERROR LOG.
2. A ROOT CAUSE section explaining the 'why'.
3. A REMEDIATION PLAYBOOK section with exact, numbered shell commands to fix the issue.
4. A VALIDATION section with a command to confirm the fix.

--- LOGS START ---
{logs}
--- LOGS END ---
"""
    headers = {"Content-Type": "application/json"}
    payload = {"model": MODEL_NAME, "prompt": prompt, "max_tokens": 512}
    
    try:
        response = requests.post(MODEL_API_URL, headers=headers, json=payload, verify=False)
        response.raise_for_status()
        result = response.json()
        if 'choices' in result and result['choices']:
            report_text = result['choices'][0].get('text', 'Error: Could not extract report.')
            print("    ✅ AI analysis complete.")
            return report_text
    except Exception as e:
        print(f"    ❌ Error querying model: {e}")
    return None

def save_report_to_json(report_text, filename="report.json"):
    """Saves the raw report text into a JSON file."""
    print(f"\n--> Step 3: Saving full report to '{filename}'...")
    try:
        report_data = {"report_content": report_text}
        with open(filename, 'w') as f:
            json.dump(report_data, f, indent=4)
        print(f"    ✅ Successfully saved full report.")
        return filename
    except Exception as e:
        print(f"    ❌ Error saving report to JSON: {e}")
    return None

def extract_and_save_keywords(report_filename="report.json", output_filename="extracted_keywords.json"):
    """Reads the full report, extracts keywords, and saves them to a new file."""
    print(f"\n--> Step 4: Extracting and normalizing keywords from '{report_filename}'...")
    try:
        with open(report_filename, 'r') as f:
            data = json.load(f)
        report_content = data.get("report_content")

        details = {}
        # Using a simpler, more direct regex pattern that is less fragile.
        host_pattern = re.search(r"AFFECTED HOST:\s*(.*)", report_content, re.IGNORECASE)
        service_pattern = re.search(r"AFFECTED SERVICE:\s*(.*)", report_content, re.IGNORECASE)

        if host_pattern:
            details["affected_host"] = host_pattern.group(1).strip()
            
        if service_pattern:
            # Normalize the extracted service name to be playbook-ready.
            raw_service_name = service_pattern.group(1).strip()
            
            # --- Normalization Logic ---
            # Rule 1: Check for common descriptive names and map them to the correct service name.
            if "apache" in raw_service_name.lower():
                normalized_service = "httpd"
            elif "nginx" in raw_service_name.lower():
                normalized_service = "nginx"
            elif "mariadb" in raw_service_name.lower():
                normalized_service = "mariadb"
            elif "mysql" in raw_service_name.lower():
                normalized_service = "mysql"
            else:
                # If no specific mapping is found, use the raw name
                normalized_service = raw_service_name

            # Rule 2: Clean up the result by removing the '.service' suffix if it exists.
            if normalized_service.endswith('.service'):
                normalized_service = normalized_service[:-len('.service')]

            details["affected_service"] = normalized_service

        if "affected_host" in details and "affected_service" in details:
            print(f"    ✅ Affected Host:    '{details['affected_host']}'")
            print(f"    ✅ Affected Service: '{details['affected_service']}'")
            
            with open(output_filename, 'w') as out_f:
                json.dump(details, out_f, indent=4)
            print(f"    ✅ Successfully saved keywords to '{output_filename}'.")
            return True
        else:
            print("    ❌ Could not extract necessary details from the report.")
            if not host_pattern:
                print("       - 'AFFECTED HOST' keyword not found in the report.")
            if not service_pattern:
                print("       - 'AFFECTED SERVICE' keyword not found in the report.")
            return False
            
    except FileNotFoundError:
        print(f"    ❌ Error: The file '{report_filename}' was not found.")
    except Exception as e:
        print(f"    ❌ An error occurred during keyword extraction: {e}")
    return False

# --- Main Execution ---
def main():
    """Main function to run the AIOps workflow."""
    # 1. Read the specified log file from MinIO
    log_content = read_log_from_minio(MINIO_BUCKET, LOG_FILE)
    if not log_content:
        sys.exit("Pipeline stopped: Could not retrieve logs.")
        
    # 2. Query the model to generate the full incident report
    incident_report = query_model_for_report(log_content)
    if not incident_report:
        sys.exit("Pipeline stopped: Could not generate AI report.")

    # 3. Save the full report to `report.json`
    report_file = save_report_to_json(incident_report)
    if not report_file:
        sys.exit("Pipeline stopped: Failed to save the report.")
        
    # 4. Extract keywords from the file just created and save to a new file
    extract_and_save_keywords(report_file)


if __name__ == "__main__":
    print("==============================================")
    print("      AIOps PIPELINE: LOGS TO KEYWORDS")
    print("==============================================")
    main()
    print("\n==============================================")
    print("            PIPELINE COMPLETE")
    print("==============================================")

      AIOps PIPELINE: LOGS TO KEYWORDS
--> Step 1: Reading 'service_error.txt' from bucket 'logs'...
    ✅ Successfully read log content from MinIO.

--> Step 2: Querying AI model for full report...


/opt/app-root/lib64/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'granite-aiops.apps.cluster-hdmxf.hdmxf.sandbox689.opentlc.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


    ✅ AI analysis complete.

--> Step 3: Saving full report to 'report.json'...
    ✅ Successfully saved full report.

--> Step 4: Extracting and normalizing keywords from 'report.json'...
    ✅ Affected Host:    'server1'
    ✅ Affected Service: 'httpd'
    ✅ Successfully saved keywords to 'extracted_keywords.json'.

            PIPELINE COMPLETE
